In [1]:
!pip install requests beautifulsoup4 selenium webdriver-manager
import requests
from bs4 import BeautifulSoup

# 请求的 URL
url = 'https://www.lawbank.com.tw/treatise/query.aspx'

# 模拟的请求 headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.lawbank.com.tw',
    'Referer': 'https://www.lawbank.com.tw/treatise/query.aspx',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
}

# 模拟的 cookies
cookies = {
    'ASP.NET_SessionId': 'fli33etnsa1phj0uurvtfvqv',  # 这是一个示例值，实际需要你捕捉到的 cookie
}

# POST 请求的表单数据
payload = {
    '__EVENTTARGET': 'ctl00$cphMain$butSubmit',
    '__EVENTARGUMENT': '',
    '__VIEWSTATE': '/wEPDwUJODM0NDY1Nzk2ZBgBBR5fX0NvbnRyb2xzUmVxdWlyZVBvc3RCYWNrS2V5X18WCgUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkMAUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkMQUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkMgUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkMwUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkNAUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkNQUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkNgUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkNwUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkOAUZY3RsMDAkY3BoTWFpbiRjYlRUc0NhdGUkOKyxjKmViv7/2++fLYr8jpjdwPJl',
    '__VIEWSTATEGENERATOR': 'E326578F',
    '__EVENTVALIDATION': '/wEdABrCtSalbZU71xETAOC72khnI7I3I1xM5ZpV+cKNPpii1I85Q0cY8nOXWEdcmC2N1XqfPkOJW0XfnjsaL7dM+WvB5OaP1FbSi8lYBJTQm15T2OGTD+X/VTUvA9VwsRjFe3bOXT7M1QIkaWq6XgWGSPjj3dJf36R25AXkFPgDm9KYs4rCjNTEPqGHrUMEyv1xVgqHAp8/Al/AuigrFDRe6HMlkHZXvjtBVOokzGQbKVID+OG6ZGf8Qv3w0wuyNI9e6fiTfl0J0r7lGjHswEyGIHKH/v+mn//T0nnZqWE3t3LT74SlDUa0SjUjwjRsn3WNaR/D02qo7xdWnwEH8X2+VYhUJ+/PsjiK7qKP52cXGRm+QYDvhThJxZC18MYuQMifLOj9pPmn7tmluCe6wHkBU0so3L4b3x3RvTU/FoX14EszrPHs86QJGdgPcrYX8oN+lg/wxylxB5y/S5t1zGHNTE4u8VYlyZtsrBIuZ7I0evWiH86+Jyes8AGbctIPPpcBpgj6GXjdWO2VQh/N6w9bJ8TSWghl0oRteUhofAdGWqfvj9Lb+1lKetZEAuNeiTP3S6tNKjK5',
    'ctl00$hidSid': '6b350d30746e46838568f8f6a8f5f0f1-30',
    'cbTTsCateAll': 'on',
    'ctl00$cphMain$cbTTsCate$0': 'on',
    'ctl00$cphMain$cbTTsCate$1': 'on',
    'ctl00$cphMain$cbTTsCate$2': 'on',
    'ctl00$cphMain$cbTTsCate$3': 'on',
    'ctl00$cphMain$cbTTsCate$4': 'on',
    'ctl00$cphMain$cbTTsCate$5': 'on',
    'ctl00$cphMain$cbTTsCate$6': 'on',
    'ctl00$cphMain$cbTTsCate$7': 'on',
    'ctl00$cphMain$cbTTsCate$8': 'on',
    'ctl00$cphMain$txtKeyword0': '借名登記',  # 搜索关键词
    'ctl00$cphMain$sltColumn0': 'T',
    'ctl00$cphMain$sltOper0': 'AND',
    'ctl00$cphMain$txtKeyword1': '',
    'ctl00$cphMain$sltColumn1': 'ALL',
    'ctl00$cphMain$sltOper1': 'AND',
    'ctl00$cphMain$txtKeyword2': '',
    'ctl00$cphMain$sltColumn2': 'ALL',
    'ctl00$cphMain$txtSY': '',
    'ctl00$cphMain$txtSM': '',
    'ctl00$cphMain$txtSD': '',
    'ctl00$cphMain$txtEY': '',
    'ctl00$cphMain$txtEM': '',
    'ctl00$cphMain$txtED': '',
    'txtmail': '',
}

# 发送 POST 请求
response = requests.post(url, data=payload, headers=headers, cookies=cookies)

# 检查请求是否成功
if response.status_code == 200:
    # 使用 BeautifulSoup 解析返回的 HTML 页面
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 提取搜索结果
    results = soup.find_all('a')  # 根据页面结构调整选择器
    for result in results:
        print(result.text)  # 打印论文标题
else:
    print(f"Request failed with status code {response.status_code}")

:::
回首頁
會員登出
購買授權
購買授權
功能介紹
預約服務
精選六法
法規查詢
司法判解
行政函釋
裁判書
起訴書
英譯法規
法學論著
法學題庫
會員專區
論著投稿


全部(110)
憲法及關係法規(1)
民事法(62)
刑事法(2)
行政法(24)
法學、法律及法制(0)
國際法(0)
大陸事務(0)
其他(0)
博碩士論文(24)
友善列印
不動產借名登記相關民事法律關係－以評析近年最高法院裁判為主
黃健彰
民事法裁判精選－借名登記之內部關係與外部關係（111 台上 2686 判決）
顏佑紘
論借名登記
蔣瑞安
借名登記股份的回復與公同共有－最高法院 110 年度台上字第 724 號民事判決
陳榮傳
原住民保留地的借名登記－大法庭裁定的商榷
陳榮傳
自耕能力與借名登記－最高法院 107 年度台上字第 1023 號判決評析
林慶郎
借名登記之舉證責任
鄭冠宇
借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定
王怡蘋
借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定
王怡蘋
原住民保留地借名登記之法律關係－兼評最高法院 108 年台上大字第 1636 號民事裁定
蔡旻耿
上一頁
下一頁
:::
法源資訊
產品服務
資料庫說明
法源徵才
交通資訊
策略聯盟
申請流程
會員條款
授權費用
付款方式
徵集論著
啟用授權專區
權利金計算說明
授權合約書下載
投稿基本資料表
使用手冊
造字檔說明
常見問題集
法令訂閱服務
金融法規自動關連AI
金融法遵外規資料服務
裁判書資料服務
訴訟案件管理系統
訂閱
取消訂閱
訂閱舊報
lawbank@lawbank.com.tw


使用規範





In [2]:
response = requests.post(url, data=payload, headers=headers, cookies=cookies)

# 打印返回的 HTML 内容，查看页面结构
if response.status_code == 200:
    print(response.text)  # 打印 HTML 响应内容
else:
    print(f"Request failed with status code {response.status_code}")



<!DOCTYPE HTML>

<html lang="zh-Hant-TW">
<head><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="CACHE-CONTROL" content="NO-CACHE" /><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta name="Keywords" content="法規、法律、行政函釋、法規新訊、裁判書、英譯法規、法學期刊、博碩士論文、法學書籍、法學題庫" /><meta name="copyright" content="本網頁著作權屬法源資訊股份有限公司所有" /><meta name="description" content="「法源法律網」為專業法學資料庫，收錄：法規（憲法、法律、法規命令、行政規則、地方法規）、司法判解（大法官解釋、司法解釋、判例、精選裁判、具參考價值裁判、決議、法律問題）、各級法院裁判書、行政函釋（解釋令函、公告、裁決、處分）、訴願決定書、法學論著全文、國家及研究所考試考古題、法律新聞…等，並每日更新各類法學資料。建置、維運「司法院法學資料檢索系統」、「裁判書查詢系統」、「全國法規資料庫」……等政府法規公開機制，為領航臺灣法學資料庫的第一品牌。" /><meta property="fb:app_id" content="846305055438782" /><meta property="og:type" content="article" /><meta property="og:image" content="https://www.lawbank.com.tw/images/logo_FBshare.png?ver=20190123" /><meta property="og:site_name" content="法源法律網" /><title>
	法源法律網-法學論著-數位法學綜合查詢結果
</title><link href="../css/reset.css" rel="stylesheet" type="text/css" media="screen" /><link href="../

# 成功抓取文章網頁和簡要資訊

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import time

# Configuration
base_url = 'https://www.lawbank.com.tw'
search_url = f'{base_url}/treatise/query.aspx'

# 模拟的请求 headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.lawbank.com.tw',
    'Referer': 'https://www.lawbank.com.tw/treatise/query.aspx',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
}

# 模拟的 cookies
cookies = {
    'ASP.NET_SessionId': 'fli33etnsa1phj0uurvtfvqv',  # 这是一个示例值，实际需要你捕捉到的 cookie
}

def build_payload(viewstate, viewstategen, eventvalidation, keyword='借名登記'):
    """建立 POST 表單 payload。"""
    return {
        "__EVENTTARGET": "ctl00$cphMain$butSubmit",
        "__EVENTARGUMENT": "",
        "__VIEWSTATE": viewstate,
        "__VIEWSTATEGENERATOR": viewstategen,       # ✅ 新增
        "__EVENTVALIDATION": eventvalidation,
        "cbTTsCateAll": "on",                       # ✅ 全選
        "ctl00$cphMain$cbTTsCate$0": "on",
        "ctl00$cphMain$cbTTsCate$1": "on",
        "ctl00$cphMain$cbTTsCate$2": "on",
        "ctl00$cphMain$cbTTsCate$3": "on",
        "ctl00$cphMain$cbTTsCate$4": "on",
        "ctl00$cphMain$cbTTsCate$5": "on",
        "ctl00$cphMain$cbTTsCate$6": "on",
        "ctl00$cphMain$cbTTsCate$7": "on",
        "ctl00$cphMain$cbTTsCate$8": "on",
        "ctl00$cphMain$txtKeyword0": keyword,       # ✅ 主關鍵詞
        "ctl00$cphMain$sltColumn0": "T",            # T=標題
        "ctl00$cphMain$sltOper0": "AND",
        "ctl00$cphMain$txtKeyword1": keyword,       # ✅ 也搜關鍵詞欄
        "ctl00$cphMain$sltColumn1": "K",            # K=關鍵詞
        "ctl00$cphMain$sltOper1": "OR",
        "ctl00$cphMain$txtKeyword2": "",
        "ctl00$cphMain$sltColumn2": "ALL",
        "ctl00$cphMain$txtSY": "",
        "ctl00$cphMain$txtSM": "",
        "ctl00$cphMain$txtSD": "",
        "ctl00$cphMain$txtEY": "",
        "ctl00$cphMain$txtEM": "",
        "ctl00$cphMain$txtED": "",
        "txtmail": "",
    }

def get_form_data(soup):
    """Extract __VIEWSTATE and __EVENTVALIDATION from HTML."""
    viewstate = soup.find('input', {'name': '__VIEWSTATE'})['value']
    eventvalidation = soup.find('input', {'name': '__EVENTVALIDATION'})['value']
    return viewstate, eventvalidation

def get_total_pages(soup):
    """Retrieve the total number of pages."""
    page_count_tag = soup.find('span', id='ctl00_cphMain_PagerBottom_lblPageCount')
    total_pages = int(page_count_tag.text) if page_count_tag else 1
    print(f"Total pages: {total_pages}")
    return total_pages





def parse_page(soup):
    """Extract all paper URLs from the page."""
    papers = soup.find_all('div', class_='TreatiseCon')
    paper_links = []
    bib_entries = []

    for paper in papers:
        link_tag = paper.find('a', href=re.compile(r'_article.aspx'))
        # 期刊論文：pl_article.aspx
        # 碩博士論文： ts_article.aspx
        # 無語
        if link_tag:
            link = base_url + link_tag['href']
            paper_links.append(link)
            print(f"Found paper link: {link}")

        bib_entry = extract_paper_info_from_list(paper)
        if bib_entry:
            bib_entries.append(bib_entry)

    return paper_links, bib_entries


def extract_paper_info_from_list(paper):
    """Extract detailed paper info and citation from each entry."""
    try:
        # 標題與連結
        title_tag = paper.find('a', id=re.compile(r'lcT_hlTitle'))
        if not title_tag:
            return ""
        title = title_tag.text.strip()
        link = base_url + title_tag['href']

        # 碩博士論文判定
        is_thesis = 'ts_article.aspx' in link

        # 作者處理
        author_info = paper.find(text=re.compile(r'作者譯者：'))
        if author_info:
            raw_text = author_info.parent.get_text(strip=True).replace('作者譯者：', '')
            guide_match = re.search(r'(.*)\s+指導\s+(.*)', raw_text)
            if guide_match:
                authors = guide_match.group(1).split('、')
                advisor = guide_match.group(2).strip()
            else:
                authors = raw_text.split('、')
                advisor = None
            author_entry = " and ".join(a.strip() for a in authors)
        else:
            author_entry = "未知"
            advisor = None

        # 論文詳細頁請求
        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
        }
        detail_resp = requests.get(link, headers=headers, timeout=10)
        detail_soup = BeautifulSoup(detail_resp.text, "html.parser")

        # 出版日期
        date_tag = detail_soup.find('th', string=re.compile('出版日期'))
        if date_tag:
            date_text = date_tag.find_next('td').get_text(strip=True)
            parts = date_text.split('.')
            year = parts[0] if len(parts) > 0 else '未知'
            month = parts[1] if len(parts) > 1 else ''
            day = parts[2] if len(parts) > 2 else ''
        else:
            year = month = day = '未知'

        # 刊登出處
        source_tag = detail_soup.find('th', string=re.compile('刊登出處'))
        source_value = source_tag.find_next('td').get_text(strip=True, separator=' ') if source_tag else '未知'

        # 叢集名稱
        collection_tag = detail_soup.find('th', string=re.compile('集叢名稱'))
        collection = collection_tag.find_next('td').get_text(strip=True) if collection_tag else '未知'

        # 頁數
        pages_tag = detail_soup.find('th', string=re.compile(r'頁\s*數'))
        pages = pages_tag.find_next('td').get_text(strip=True) if pages_tag else '未知'

        # ✅ 出版社（允許中間有空白）
        publisher_tag = detail_soup.find('th', string=re.compile(r'出\s*版\s*社'))
        publisher = publisher_tag.find_next('td').get_text(strip=True) if publisher_tag else '未知'

        # ✅ ISBN（同樣容錯）
        isbn_tag = detail_soup.find('th', string=re.compile(r'I\s*S\s*B\s*N'))
        isbn = isbn_tag.find_next('td').get_text(strip=True) if isbn_tag else '無'

        # 點閱次數
        views_tag = detail_soup.find('span', id=re.compile(r'lblReadCount'))
        views = views_tag.get_text(strip=True) if views_tag else '未知'

        # 文獻引用內容
        citation_tag = detail_soup.select_one(".citation-content")
        citation_text = citation_tag.get_text(strip=True) if citation_tag else "未找到文獻引用"

        # BibTeX 類型與組裝
        bib_type = "thesis" if is_thesis else "article"
        advisor_entry = f"\n  advisor = {{{advisor}}}," if advisor else ""

        bib_entry = (
            f"@{bib_type}{{{re.sub(r'\\W+', '_', title)},\n"
            f"  title = {{{title}}},\n"
            f"  author = {{{author_entry}}},\n"
            f"  year = {{{year}}},\n"
            f"  month = {{{month}}},\n"
            f"  day = {{{day}}},\n"
            f"  journal = {{{source_value}}},\n"
            f"  series = {{{collection}}},\n"
            f"  pages = {{{pages}}},\n"
            f"  publisher = {{{publisher}}},\n"
            f"  isbn = {{{isbn}}},\n"
            f"  note = {{點閱次數：{views}}},{advisor_entry}\n"
            f"  howpublished = {{{citation_text}}},\n"
            f"  url = {{{link}}}\n"
            f"}}"
        )
        print(f"✅ 取得：{title}")
        return bib_entry

    except Exception as e:
        print(f"❌ Error extracting {title if 'title' in locals() else '未知'}: {e}")
        return ""


def get_form_data(soup):
    """Extract critical form fields."""
    def extract_value(name):
        tag = soup.find('input', {'name': name})
        return tag['value'] if tag else ''

    viewstate = extract_value('__VIEWSTATE')
    eventvalidation = extract_value('__EVENTVALIDATION')
    viewstateguid = extract_value('__VIEWSTATEGUID')
    # eventtarget = extract_value('__EVENTTARGET')
    # eventargument = extract_value('__EVENTARGUMENT')
    ctl00_hidSid = extract_value('ctl00$hidSid')

    return viewstate, eventvalidation, viewstateguid, ctl00_hidSid
            # , eventtarget, eventargument

def main():
    with requests.Session() as session:
        # Initial request to get the form data
        # response = session.post(search_url, headers=headers, cookies=cookies)
        response = session.post(search_url, headers=headers)
        if response.status_code != 200:
            print(f"Initial request failed: {response.status_code}")
            return

        soup = BeautifulSoup(response.content, 'html.parser')

        viewstate, eventvalidation, viewstateguid, ctl00_hidSid = get_form_data(soup)

        payload = build_payload(viewstate, viewstateguid, eventvalidation)
        response = session.post(search_url, headers=headers, data=payload)
        if response.status_code != 200:
            print(f"初始請求失敗: {response.status_code}")
            return
        
        soup = BeautifulSoup(response.content, 'html.parser')
        # print(soup)

        viewstate, eventvalidation, viewstateguid, ctl00_hidSid = get_form_data(soup)
        total_pages = get_total_pages(soup)

        all_links = []
        all_bib_entries = []

        for page_num in range(1, total_pages + 1 ):
            print(f"current page num: {page_num}")

            if page_num > 1:
                # time.sleep(4)

                response = session.post("https://www.lawbank.com.tw/treatise/queryresult.aspx", headers=headers, data=payload)
                # print(f"payload: {payload}")
                soup = BeautifulSoup(response.content, 'html.parser')
                # print(soup)
                viewstate, eventvalidation, viewstateguid, ctl00_hidSid = get_form_data(soup)

            links, entries = parse_page(soup)
            all_links.extend(links)
            all_bib_entries.extend(entries)
            
            # print(links)
            time.sleep(1)

            # next Page payload
            payload = {
                '__EVENTTARGET': 'ctl00$cphMain$PagerBottom$butNext',
                '__EVENTARGUMENT': '',
                '__LASTFOCUS': '',
                '__VIEWSTATEGUID': viewstateguid,
                '__VIEWSTATE': viewstate,
                '__EVENTVALIDATION': eventvalidation,
                'ctl00$hidSid':ctl00_hidSid,
                'ctl00$cphMain$ssSearch$txtKeyword': '',
                'ctl00$cphMain$ssSearch$cbItems$0': 'on',
                'ctl00$cphMain$ssSearch$cbItems$1': 'on',
                'ctl00$cphMain$ssSearch$cbItems$2': 'on',
                'ctl00$cphMain$ssSearch$cbItems$3': 'on',
                'ctl00$cphMain$ssSearch$cbItems$4': 'on',
                'ctl00$cphMain$ddlSort': 'F',
                'ctl00$cphMain$PagerBottom$ddlPage': str(page_num),
                'txtmail': '',
            }

        with open('paper_links.txt', 'w', encoding='utf-8') as f:
            f.write('\n'.join([link for link in all_links]))
        with open('papers.bib', 'w', encoding='utf-8') as bib_file:
            bib_file.write('\n\n'.join(all_bib_entries))

        print("所有文章的 BibTeX 已成功儲存到 papers.bib")
        # print(all_links)


if __name__ == "__main__":
    main()

Total pages: 4
current page num: 1
Found paper link: https://www.lawbank.com.tw/treatise/pk_article.aspx?AID=A000004483
✅ 取得：不動產借名登記相關民事法律關係－以評析近年最高法院裁判為主
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000265709


/var/folders/kx/yl3w6mcd1g7767kzzplxnd380000gn/T/ipykernel_40914/1683390043.py:113: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author_info = paper.find(text=re.compile(r'作者譯者：'))


✅ 取得：民事法裁判精選－借名登記之內部關係與外部關係（111 台上 2686 判決）
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000260400
✅ 取得：論借名登記
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000259825
✅ 取得：借名登記股份的回復與公同共有－最高法院 110 年度台上字第 724 號民事判決
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000255086
✅ 取得：原住民保留地的借名登記－大法庭裁定的商榷
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000254978
✅ 取得：自耕能力與借名登記－最高法院 107 年度台上字第 1023 號判決評析
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000254498
✅ 取得：借名登記之舉證責任
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000254768
✅ 取得：借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定
Found paper link: https://www.lawbank.com.tw/treatise/pl_article.aspx?AID=P000254804
✅ 取得：借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定
Found paper link: https://www.lawbank.com.tw/treatise/dt_article.aspx?AID=D000024066
✅ 取得：原住民保留地借名登記之法律關係－兼評最高法院 108 年台上大字第 1636 號民事裁定


                
    #             翻頁按鈕的完整payload:

    # __EVENTTARGET: ctl00$cphMain$PagerBottom$butNext
    # __EVENTARGUMENT: 
    # __LASTFOCUS: 
    # __VIEWSTATEGUID: 80a2d3b8-e5c8-43d9-b203-a487ae972d98
    # __VIEWSTATE: 
    # __EVENTVALIDATION: /wEdABmvVXD1oYELeveMr0vHCmYPI7I3I1xM5ZpV+cKNPpii1KXFjvDThGtzxdBq0qG1F8MrYZQpWHrNDbSvGKNEjRbJCYSTjMa5Wou+X1yoORJHHRVQ/X0N2uOSAOe3j7bygztRa8F4n1gZX1q/MDYCRKnQJr+LWnYXiepNn9LOJTLwc/U/+1yGAU/vIrRjN3PpN36kwsVrTE5lI3m/jaN/Z7agGLuKNZoJ63ze7nptA/DmwStHBDVp1ATCE9CUraMjugmVrChBgGEiJLk6eEVknE5VRwzwLsOC0ztH96Z+Pjc3bcaqtpGHtJoXe8S64bfu3HyZCVQPU4a3e5L/CG3LeAtVwwKs/ETf+jQS0rGO+SSPyioamkV2E2K+hlE5ZkDdRlcMKW8ABBw3EMZHGTYPJhPL+2XWd7qmJVj5Dx877yczVubJhP8n9zs96SiSGHl57SMEEvLWBkT2zRYR68BbzBgi0ulGuCmJgc7C1KRIjuMjj/vNSQa9ABodES+dY+b9Ax5aSg7OlsQWsZ3//ngvby0KLhA89IbDHYEKyWk1mbhSN/FcTQM=
    # ctl00$hidSid: 82b353dd85a44d6fa057785302199527-30
    # ctl00$cphMain$ssSearch$txtKeyword: 
    # ctl00$cphMain$ssSearch$cbItems$0: on
    # ctl00$cphMain$ssSearch$cbItems$1: on
    # ctl00$cphMain$ssSearch$cbItems$2: on
    # ctl00$cphMain$ssSearch$cbItems$3: on
    # ctl00$cphMain$ssSearch$cbItems$4: on
    # ctl00$cphMain$ddlSort: F
    # ctl00$cphMain$PagerBottom$ddlPage: 1
    # txtmail: 


下載

In [4]:
import os
import re
import time
import random
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, UnexpectedAlertPresentException
)
from webdriver_manager.chrome import ChromeDriverManager


# 定義下載目錄：以目前工作目錄為基礎建立 "downloaded_papers" 資料夾
DOWNLOAD_FOLDER = os.path.join(os.getcwd(), 'downloaded_papers_1014')
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)





def parse_bib_file(filename='papers.bib'):
    """從 BibTeX 文件中提取文章的標題、作者與 URL。"""
    titles = []
    urls = []
    authors = []

    current_title = None
    current_url = None
    current_author = None

    with open(filename, 'r', encoding='utf-8') as f:
        for raw_line in f:
            line = raw_line.strip()
            if not line:
                continue
            if line.startswith('@'):
                current_title = None
                current_url = None
                current_author = None
                continue
            if line.startswith('title ='):
                current_title = line.split('{', 1)[1].rsplit('}', 1)[0]
            elif line.startswith('author ='):
                current_author = line.split('{', 1)[1].rsplit('}', 1)[0]
            elif line.startswith('url ='):
                current_url = line.split('{', 1)[1].rsplit('}', 1)[0]
            elif line.startswith('}'):
                if current_title and current_url:
                    author_value = current_author if current_author else '未知作者'
                    titles.append(current_title)
                    urls.append(current_url)
                    authors.append(author_value)
                current_title = None
                current_url = None
                current_author = None

    return titles, urls, authors



def setup_driver():
    """初始化 Selenium 瀏覽器驅動，並設置自定義下載目錄."""
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # 無頭模式（若需要顯示瀏覽器，則註解掉這行）
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    prefs = {
        "download.default_directory": DOWNLOAD_FOLDER,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(
        service=webdriver.chrome.service.Service(ChromeDriverManager().install()),
        options=options,
    )
    return driver



def handle_multiple_alerts(driver, final_click_element_locator=None):
    """檢測並自動接受多個連續彈窗，在最後一個彈窗後再次點擊按鈕（如果需要）。"""
    alert_detected = False
    try:
        while True:
            WebDriverWait(driver, 2).until(EC.alert_is_present())
            alert = Alert(driver)
            print(f"彈窗內容：{alert.text}")
            time.sleep(2)
            alert.accept()
            print("彈窗已接受")
            alert_detected = True
            time.sleep(2)
    except TimeoutException:
        print("沒有更多的彈窗需要處理")

    if alert_detected and final_click_element_locator:
        try:
            print("再次查找並點擊按鈕...")
            final_click_element = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable(final_click_element_locator)
            )
            final_click_element.click()
        except Exception as e:
            print(f"再次點擊按鈕失敗：{e}")



def sanitize_filename(text):
    """移除檔名中不允許的字元並壓縮空白。"""
    sanitized = re.sub(r'[\\/:*?"<>|]', '', text)
    sanitized = sanitized.replace("\n", " " ).replace("\r", " ")
    sanitized = re.sub(r'\s+', ' ', sanitized).strip()
    return sanitized or '未命名'




def rename_file(temp_filename, title, author):
    """將最新下載的檔案重命名為「作者 - 標題」格式。"""
    try:
        safe_author = sanitize_filename(author)
        safe_title = sanitize_filename(title)
        new_name = os.path.join(DOWNLOAD_FOLDER, f"{safe_author} - {safe_title}.pdf")
        temp_path = os.path.join(DOWNLOAD_FOLDER, temp_filename)

        if os.path.exists(new_name):
            print(f"文件已存在，跳過：{new_name}")
            os.remove(temp_path)
        else:
            os.rename(temp_path, new_name)
            print(f"文件已重命名為：{new_name}")
    except Exception as e:
        print(f"重命名時發生錯誤：{e}")



def wait_for_download(timeout=60):
    """監控下載目錄，確認是否有新的 PDF 文件下載完成。"""
    start_time = time.time()
    before_download = set(os.listdir(DOWNLOAD_FOLDER))

    while True:
        current_files = set(os.listdir(DOWNLOAD_FOLDER))
        new_files = current_files - before_download

        if new_files:
            new_file = new_files.pop()
            new_file_path = os.path.join(DOWNLOAD_FOLDER, new_file)
            if os.path.getsize(new_file_path) > 0 and not new_file.endswith(('.crdownload', '.part')):
                print(f"下載完成：{new_file}")
                return new_file

        if time.time() - start_time > timeout:
            print("下載超時")
            return None

        time.sleep(1)



def download_pdf(driver, paper_url, title, author):
    """進入每篇論文網站，嘗試點擊下載按鈕並保存 PDF。"""
    try:
        driver.get(paper_url)

        try:
            download_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, "ctl00_cphMain_adlPoint_imgDL"))
            )
            download_button.click()
            handle_multiple_alerts(driver, (By.ID, "ctl00_cphMain_adlPoint_imgDL"))
        except TimeoutException:
            print(f"該論文無公開電子全文，跳過：{title}")
            return

        driver.find_element(By.ID, "cbagree").click()
        driver.find_element(By.ID, "btnDL").click()

        downloaded_file = wait_for_download()
        if downloaded_file:
            rename_file(downloaded_file, title, author)
        else:
            print(f"下載失敗或超時：{title}")

        time.sleep(random.uniform(3, 5))

    except Exception as e:
        print(f"下載失敗：{e}")



def main(download_all=False, start_index=0, amount=5):
    """根據參數選擇下載全部或部分論文。"""
    titles, urls, authors = parse_bib_file()

    if download_all:
        start_index = 0
        end_index = len(titles)
    else:
        end_index = min(start_index + amount, len(titles))

    driver = setup_driver()

    for title, url, author in zip(
        titles[start_index:end_index],
        urls[start_index:end_index],
        authors[start_index:end_index],
    ):
        print(f"正在嘗試下載：{author} - {title}")
        download_pdf(driver, url, title, author)

    driver.quit()


if __name__ == "__main__":
    # main(download_all=False, start_index=10, amount=3)
    main(download_all=True)


正在嘗試下載：黃健彰 - 不動產借名登記相關民事法律關係－以評析近年最高法院裁判為主
下載失敗：Message: element click intercepted: Element <input type="image" name="ctl00$cphMain$adlPoint$imgDL" id="ctl00_cphMain_adlPoint_imgDL" src="../images/icon_downloadAll.gif" align="absmiddle" onclick="return LoginAlert();" style="border-width:0px;"> is not clickable at point (345, 483). Other element would receive the click: <div class="tmask" id="" style="height: 1522px; width: 1200px; opacity: 0.5; display: block;"></div>
  (Session info: chrome=141.0.7390.108); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
0   chromedriver                        0x000000010066bcf8 cxxbridge1$str$ptr + 2895872
1   chromedriver                        0x0000000100663c34 cxxbridge1$str$ptr + 2862908
2   chromedriver                        0x0000000100189570 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74324
3   chromedri

In [5]:
def parse_bib_file(filename='papers.bib'):
    """從 BibTeX 文件中提取文章的標題、作者與 URL。"""
    titles = []
    urls = []
    authors = []

    current_title = None
    current_url = None
    current_author = None

    with open(filename, 'r', encoding='utf-8') as f:
        for raw_line in f:
            line = raw_line.strip()
            if not line:
                continue
            if line.startswith('@'):
                current_title = None
                current_url = None
                current_author = None
                continue
            if line.startswith('title ='):
                current_title = line.split('{', 1)[1].rsplit('}', 1)[0]
            elif line.startswith('author ='):
                current_author = line.split('{', 1)[1].rsplit('}', 1)[0]
            elif line.startswith('url ='):
                current_url = line.split('{', 1)[1].rsplit('}', 1)[0]
            elif line.startswith('}'):
                if current_title and current_url:
                    author_value = current_author if current_author else '未知作者'
                    titles.append(current_title)
                    urls.append(current_url)
                    authors.append(author_value)
                current_title = None
                current_url = None
                current_author = None

    return titles, urls, authors


def calculate_unique_articles(titles):
    """根據標題計算不重複的文章數量。"""
    unique_titles = set()
    duplicate_count = 0

    for title in titles:
        if title in unique_titles:
            duplicate_count += 1
        else:
            unique_titles.add(title)

    print(f"總文章數：{len(titles)}")
    print(f"不重複的文章數：{len(unique_titles)}")
    print(f"重複的文章數：{duplicate_count}")


def main():
    titles, urls, authors = parse_bib_file('papers.bib')
    calculate_unique_articles(titles)


if __name__ == "__main__":
    main()


總文章數：35
不重複的文章數：31
重複的文章數：4


技術要點總結

完整爬蟲技術要點總結

該爬蟲實現了從 學術網站（如 LawBank）自動檢索和下載多篇論文的全流程，包括 表單提交、自動化網頁解析、文件下載 及 檔案管理。以下是整個爬蟲的主要技術要點：

1. HTTP 表單模擬提交

	•	技術細節：
	•	使用 requests 庫模擬網頁的 POST 請求，向服務器提交查詢關鍵字。
	•	利用表單中 __VIEWSTATE 和 __EVENTVALIDATION 等動態參數，模擬複雜的 ASP.NET 表單請求。
	•	技術要點：
	•	伺服器驗證與狀態管理：需要提取頁面中的隱藏欄位（如 __VIEWSTATE）以維持伺服器狀態。
	•	Cookies 模擬：透過 cookies 模擬使用者會話。

2. HTML 解析與資料提取

	•	使用工具：BeautifulSoup 進行網頁解析。
	•	步驟：
	•	在查詢結果頁面中解析每篇論文的 標題、URL、作者及指導教授 等信息。
	•	區分 期刊論文 和 碩博士論文，並根據不同文章類型生成 BibTeX 條目。
	•	技術要點：
	•	正則表達式 用於匹配特定的 HTML 標籤和內容（如「作者譯者：…指導」）。
	•	提取動態頁面中的資料，如多頁數據及其每頁上的論文資訊。

3. 多頁查詢處理與資料累積

	•	在 main() 函數中透過 遞增頁碼 進行多頁的查詢結果解析。
	•	使用每頁的 HTML 內容來解析所有論文的 URL 和 BibTeX 條目。
	•	技術要點：
	•	避免超時：每次解析頁面之間使用 time.sleep() 進行延遲。
	•	儲存所有的 URL 和 BibTeX 條目，並輸出到 papers.bib 文件。

4. Selenium 自動化操作與下載控制

	•	使用 Selenium 驅動 Chrome 瀏覽器 進行自動化操作，如進入論文頁面、點擊下載按鈕。
	•	配置了自動下載路徑，以避免每次都彈出保存對話框。
	•	技術要點：
	•	使用 webdriver-manager 自動安裝並管理 Chrome 驅動。
	•	支援多重彈窗處理，在下載前後自動接受所有彈窗。

5. 處理多重彈窗與條款同意

	•	使用 handle_multiple_alerts() 函數檢測並自動接受多個彈窗。
	•	若檢測到最後的彈窗，需再次點擊下載按鈕，確保文件下載成功。
	•	技術要點：
	•	彈窗處理：使用 WebDriverWait 和 alert.accept() 來處理動態彈窗。
	•	再次點擊：若有彈窗，則在所有彈窗處理完成後再次點擊下載按鈕。

6. 監控 PDF 檔案下載

	•	在 wait_for_download() 函數中持續監控下載目錄，確保檔案已完整下載。
	•	技術要點：
	•	輪詢檢測：每秒檢查下載目錄中的新文件，並確認其大小是否大於 0。
	•	檢查 臨時文件（如 .crdownload/.part） 是否存在，確保文件已下載完成。

7. 檔案重命名與重複檢查

	•	使用 rename_file() 函數將下載的文件重命名為對應的論文標題，避免重複檔案。
	•	技術要點：
	•	若檔案重複，則刪除新下載的重複文件。
	•	確保所有下載的文件名稱清晰且唯一。

8. 切片與全量下載控制

	•	在 main() 函數中實現了 切片下載 和 全量下載 兩種模式：
	•	使用 download_all 參數控制是下載全部文章還是僅下載部分文章。
	•	可通過 start_index 和 amount 控制要下載的起始點和數量。

9. 程式結構與控制流程

	•	使用 if __name__ == "__main__" 來定義程式的執行入口。
	•	根據使用者的需求選擇下載全部文件或特定範圍內的文件。

完整技術要點概述

技術要點	關鍵點
HTTP 表單提交	模擬複雜的 ASP.NET 表單提交，使用隱藏欄位維護狀態。
HTML 解析與資料提取	使用 BeautifulSoup 解析論文的標題、作者和 URL。
多頁查詢處理	支援多頁的查詢結果解析，避免漏掉任何論文。
自動化瀏覽器操作	使用 Selenium 自動化操作，點擊按鈕並進行下載。
多重彈窗處理	自動檢測並接受多個彈窗，確保下載流程順利。
PDF 下載監控	每秒監控下載目錄，確認文件是否下載完成。
檔案重命名	自動重命名下載的文件，避免重複和名稱混淆。
隨機等待時間	模擬人為操作，避免被網站識別為機器人。
切片與全量下載控制	支援部分或全部文件下載，提升靈活性。

結論

這個爬蟲項目展示了多個關鍵技術的綜合應用，包括 HTTP 表單模擬、HTML 解析、網頁自動化、彈窗處理、PDF 下載監控 及 檔案管理。其設計具有高效能、穩定性和可擴展性，適用於處理大批量的學術資料下載。

此爬蟲的結構化和模組化設計，使其符合良好的程式設計原則，並且能輕鬆應對未來的維護與功能擴充需求。